We recommend using the EduCortex web viewer (http://paulscotti.com/educortex) rather than creating a local server if you simply wish to try out the tool or to use it for educational purposes. We provide the below code in case you want to understand how EduCortex was created or if you want to contribute to this project.

We use Pycortex as the base for our 3D viewer, where we overlay maps on a fsaverage surface from Freesurfer. See https://github.com/gallantlab/pycortex for more information on Pycortex. We added some extra buttons and menus to the viewer using HTML/CSS/JavaScript, which are contained in the my_template.html and index.html files.

We use the Neurosynth Python package as the source of our data (i.e., we import meta-analysis maps from Neurosynth). See https://github.com/neurosynth/neurosynth or https://neurosynth.org/ for more information on Neurosynth. 

Below we detail the following steps:

° initialize Pycortex with the fsaverage brain, using my_template.html as a web template  

° load PCA components across all meta-analysis maps (see PCA/PCA_extraction.ipynb for how components were created) and then overlay the top components on fsaverage 

° create png files for every term's meta-analysis map on Neurosynth, which can then be loaded onto the fsaverage brain in the viewer

° weight each term with each PCA component to construct the PCA wordcloud used in EduCortex  

° load Glasser atlas (Glasser et al., 2016) annotation files to overlay anatomical labels on the brain

# Setup

In [55]:
import os
import numpy as np
from matplotlib import pyplot as plt
import pandas
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, MinMaxScaler # to scale PCA scores to be between 0 and 255, for visualization
import cortex
from cortex.webgl.data import Package
from neurosynth import meta, decode, network, Dataset
import nibabel as nib
from scipy import spatial # to extract the convex hull
import pickle
def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)
def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))
class MacOSFile(object):
    def __init__(self, f):
        self.f = f
    def __getattr__(self, item):
        return getattr(self.f, item)
    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)
    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size
%matplotlib inline

 # Initialize fsaverage brain with top 3 PCA components

In [15]:
comps = np.load('PCA/components.npz')['arr_0']
comps = comps.transpose(0,3,2,1)
comps.reshape(10,-1).shape

(10, 902629)

In [76]:
def make_colorvol(proj, meanstds, rgbpcs, mask, flips=tuple(), clip_lim=3):
    rgbdata = proj[rgbpcs,:]
    for f in flips:
        rgbdata[f] *= -1
    zrgb = (rgbdata.T / meanstds).T
    #zrgb = npp.rs(rgbdata.T).T
    crgb = np.clip(zrgb, -clip_lim, clip_lim) ## clip to 3 standard deviations
    srgb = crgb/clip_lim/2.0 + 0.5 ## scale to 0..1
    
    rgbvol = np.zeros([3]+list(mask.shape))
    rgbvol[0] = vox_to_mask(srgb[0], mask)
    rgbvol[1] = vox_to_mask(srgb[1], mask)
    rgbvol[2] = vox_to_mask(srgb[2], mask)

    colorvol = (rgbvol*255).astype(np.uint8)
    return colorvol.transpose(1,2,3,0)

def vox_to_mask(data, mask):
    dvol = mask.copy().astype(np.float)
    dvol[mask>0] = data
    return dvol

In [77]:
rescale = lambda x: x/x.std()

In [78]:
flat_comps = comps.reshape(10,-1)

colorvol = make_colorvol(flat_comps, flat_comps.std(1)[:3], [0,1,2], np.ones(comps[0].shape), clip_lim=4)

In [79]:
rgbvol = cortex.dataset.normalize((colorvol, 'fsaverage', 'atlas_2mm'))
rgbvol.alpha.vmin = 0

In [80]:
p = Package(rgbvol)
png = list(p.images.values())[0][0]
with open(("rgbvol1.png"),"wb") as f:
    f.write(png)

In [49]:
# open new tab that displays the PCA brain
cortex.webgl.show(data=rgbvol, template='my_template.html', labels_visible=(""), overlays_visible=(""), recache=True)

## or create local server:
# cortex.webgl.make_static("static", data=rgbvol, template='my_template.html', labels_visible=(""), overlays_visible=(""), recache=True)

# Create png for every neurosynth term to then project onto fsaverage

In [60]:
# see https://github.com/neurosynth/neurosynth if you encounter errors with Dataset
dataset = Dataset('neurosynth-data/database.txt')
dataset.add_features('neurosynth-data/features.txt')
terms = pandas.read_csv('neurosynth-data/analysis_filter_list.tsv',delimiter='\t')
kept_terms = terms['term'][terms['keep']==1]
np.savez("neurosynth_terms",'kept_terms')
for term in kept_terms:
    ids = dataset.get_studies(term)
    ma = meta.MetaAnalysis(dataset, ids)
    maps[term] = ma.images['association-test_z']
    ma.save_results(os.path.join('neurosynth-data','maps',term))

In [ ]:
for i in range(concat_maps.shape[0]):
    
    term = kept_terms.iloc[i];

    p = Package(cortex.Volume(("term_maps/"+term+".nii"), subject, 'atlas_2mm'))

    png = list(p.images.values())[0][0]

    with open(("neurosynth-maps/"+term+".png"),"wb") as f:
        f.write(png)

# Associate terms with PCA components

In [29]:
## note: concat_maps.p was too large to upload to GitHub, see PCA/PCA_extraction.ipynb to create concat_maps.p
concat_maps = pickle_load("PCA/concat_maps.p")
ncomp = 10
pca = PCA(copy=True, iterated_power='auto', n_components=ncomp, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=False).fit(concat_maps)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)

[0.10724451 0.07186315 0.04695774 0.03718591 0.03011859 0.02566107
 0.01888798 0.01386358 0.01315753 0.01177422]
[7371.67572625 6034.36808339 4877.89306864 4340.78108748 3906.57327303
 3605.9198214  3093.65168395 2650.43018478 2582.0570494  2442.55709004]


In [30]:
# this is a list of all neurosynth terms
terms = pandas.read_csv('PCA/analysis_filter_list.tsv',delimiter='\t')
# but we're only interested in the terms that are non-anatomical
terms_anatfilter = pandas.read_csv('PCA/neurosynth_terms_anatfilter.txt',delimiter='\t')

# only keep terms that have a 'keep' value of 1 and n 'anatomical' value of 0
kept_terms_anatfilter = terms_anatfilter['term'][np.logical_and(terms_anatfilter['keep']==1, terms_anatfilter['anatomical']==0)]
# we will take these terms and find their location in a list that is ordered alphabetically
kept_terms = terms['term'][terms['keep']==1]
anatfilter_indices = [i for i,k in enumerate(list(kept_terms)) if k in list(kept_terms_anatfilter)]
kept_terms_anatfilter = kept_terms.iloc[anatfilter_indices]

concat_maps_anatfilter = concat_maps[anatfilter_indices]

In [38]:
# see PCA/PCA_extraction.ipynb to create components.npz
components = np.load("PCA/components.npz")

In [ ]:
# create PCA component png maps
for i in range(10):
    p = Package(cortex.Volume((components[i,:,:,:].T/np.std(components[i,:,:,:].T)), subject, 'atlas_2mm'))

    png = list(p.images.values())[0][0]

    with open(("neurosynth-maps/pca"+str(i+1)+".png"),"wb") as f:
        f.write(png)

In [39]:
term_weights = pca.transform(concat_maps_anatfilter)
# term_weights = np.load("PCA/term_weights.npz")['arr_0']

In [40]:
for i in range(ncomp):
    a = term_weights[:,i].argsort()[::-1];
    print('\n',i);
    for j in range(5):
        print(kept_terms_anatfilter.iloc[a[j]]);
        print(term_weights[a[j],i]);
    print('\n')
    for k in range(5):
        print(kept_terms_anatfilter.iloc[a[-(k+1)]]);
        print(term_weights[a[-(k+1)],i]);


 0
motor
1317.6320905494947
movements
1082.6326032578986
movement
900.2029858276059
hand
887.2103314241627
finger
861.8239098596807


emotional
-608.1144302905019
negative
-514.7500530933149
reward
-500.44840115901064
social
-465.58541684155347
neutral
-448.30683321309544

 1
semantic
633.6148278818989
language
629.3179671445515
words
615.9370102630622
visual
603.6445620059216
word
595.8302894672337


pain
-605.0312709266811
motor
-578.627686885328
reward
-496.6405797600661
finger
-444.48396863699276
movement
-434.6616056838099

 2
auditory
813.2986946934237
speech
679.4845842781803
sounds
590.4998321348987
listening
517.4453533388773
acoustic
465.6533403985085


memory
-623.8525727947683
task
-589.5702800580941
retrieval
-466.03949697898537
working memory
-452.0013094077859
working
-448.10586096960776

 3
visual
383.39499851662646
motion
356.8537640756567
object
310.57966394226366
spatial
282.95892195528666
faces
277.5914854709835


auditory
-413.6016425246062
semantic
-404.197738839

In [ ]:
# Create wordclouds
first_three = term_weights[:,0:2]
for i in range(3):

    hull = spatial.ConvexHull(term_weights[:, i*3:(i+1)*3])
    vertices = first_three[hull.vertices,:]

    pca = PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
      svd_solver='randomized', tol=0.0, whiten=False).fit(vertices)
    twod_projection = pca.transform(vertices)
    twod_projection

    scaler = MinMaxScaler(feature_range=(0, 0.99))
    scaler = scaler.fit(term_weights[:, i*3:(i+1)*3])
    colors = scaler.transform(term_weights[hull.vertices, i*3:(i+1)*3])
    colors = colors+.3 # make colors a bit brighter so it can be read on black bg
    colors[colors>1] = 1

    fig, ax = plt.subplots()

    # twod_projection[:,0].shape
    ax.scatter(twod_projection[:,0], twod_projection[:,1], c='black')

    # fig.patch.set_visible(False)
    ax.axis('off')
    for j in range(twod_projection.shape[0]):
        ax.text(twod_projection[j,0],twod_projection[j,1],kept_terms_anatfilter.iloc[hull.vertices[j]], color = colors[j,:], fontweight='bold')

    plt.savefig('PCA/wordCloud%d_anatfilter.png'%(i+1), format='png', transparent=True, dpi=300)

## Show anatomical labels

In [62]:
# load Glasser atlas
lh_aparc_file = os.path.join('atlases','lh.HCP-MMP1.annot')
rh_aparc_file = os.path.join('atlases','rh.HCP-MMP1.annot')
lpinds, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file)
lpinds_orig, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file, True)
lpinds[lpinds_orig==0] = -1
rpinds, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file)
rpinds_orig, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file, True)
rpinds[rpinds_orig==0] = -1

aparc = cortex.Vertex(np.hstack([lpinds, rpinds]),'fsaverage')

## open new tab to display brain:
# cortex.webgl.show(data=rgbvol, template='my_template.html', overlays_visible=("rois"), recache=True)

## or create local server:
cortex.webgl.make_static("static", data=rgbvol, template='my_template.html', labels_visible=(""), overlays_visible=("rois"), recache=True)

# Additional notes

° You will need to have your functional map .pngs located in the data folder in your static local server for the functional maps for every term to be displayed on the brain correctly. If the path is not accessible, you may also need to edit the code of the index.html file inside the same folder to ensure that your server has access to your local files.

° There are some changes in the index.html file on github.com/PaulScotti/educortex that add a few additional features, such as a welcome pop-up window, making the top-left panel non-clickable, etc. You can copy this index.html file and modify it to your own use.